In [18]:
# Import dependencies
import pandas as pd
import numpy as np
import requests

from citipy import citipy
from config import weather_api_key
from datetime import datetime

In [19]:
# Create a set of 2000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs = zip(lats, lngs)
lats_lngs

In [20]:
# Add latitudes and longitudes to a list
coordinates = list(lats_lngs)
coordinates

[(-84.29393009846363, 158.41119998870556),
 (-35.65820109563413, -20.48980202831828),
 (14.449212172310737, -132.67536984228659),
 (26.499569270547724, 63.84872282594466),
 (65.3075069903366, 150.41517813126154),
 (35.86448812021621, 155.86836821452425),
 (58.19859449265951, -57.35937014781747),
 (-9.472213925395337, 136.3661455309549),
 (-59.86103954909706, 63.1857878957768),
 (43.626979214205704, 45.332997877231946),
 (-89.18240954899133, 34.283986822831025),
 (-48.122834748611744, 138.26592750178952),
 (-66.65270822171914, 130.57677698978245),
 (-3.889396932527106, -44.73620096217266),
 (61.53468554597123, -148.73804028247412),
 (-56.62157979262211, 108.64874527636698),
 (46.20794938109108, -105.5199979489806),
 (-51.5466753570513, 19.903082206022276),
 (-9.48280741889549, 34.22164274327062),
 (15.959455660631676, -110.92716726888627),
 (-31.995521818746603, 29.967240528250983),
 (43.76588997329307, 162.1356149273297),
 (-29.37458785327864, 16.369940682842383),
 (66.55556413939189, 

In [21]:
# Get the nearest city using the citipy module
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique then add it to the cities list
    if city not in cities:
        cities.append(city)

# Check count to confirm sufficient cities
len(cities)

730

In [22]:
# Build URL for Weather API call using my weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [23]:
# Test if URL works
test_city_url = url + "&q=" + "Boston"
test_city_weather = requests.get(test_city_url)
test_city_weather

<Response [200]>

In [24]:
# Get the JSON text of the "get" request
test_city_weather.json()

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 52.43,
  'feels_like': 49.91,
  'temp_min': 45.48,
  'temp_max': 59.38,
  'pressure': 1020,
  'humidity': 54},
 'visibility': 10000,
 'wind': {'speed': 8.05, 'deg': 100},
 'clouds': {'all': 40},
 'dt': 1651358668,
 'sys': {'type': 2,
  'id': 2009921,
  'country': 'US',
  'sunrise': 1651311643,
  'sunset': 1651362119},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [ ]:
# Create an empty list to hold the weather data
weather_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the cities list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    
    # Log the URL, record, and set numbers and the city
    print(f"Processing record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1

# Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append city weather information into weather_data list
        weather_data.append({"City": city.title(), 
                             "Country": city_country,
                             "Date": city_date,
                             "Lat": city_lat,
                             "Lng": city_lng,
                             "Max Temp": city_max_temp,
                             "Humidity": city_humidity,
                             "Cloudiness": city_clouds,
                             "Wind Speed": city_wind,
                             "Current Description": city_weather_description})        

# If an error is experience, skip the city
    except:
        print("City not found. Skipping...")
        pass       
        
# Indicate that Data Retrieval is complete.
print("--------------------------------")
print("Data Retrieval Complete.        ")
print("--------------------------------")

In [ ]:
# Convert the array of dictionaries to a Pandas DataFrame
weather_data_df = pd.DataFrame(weather_data)
weather_data_df.head(10)

In [27]:
# Create the output file (CSV)
output_data_file = "Weather_Database/WeatherPy_challenge.csv"

# Export the weather_data DataFrame into a CSV file
weather_data_df.to_csv(output_data_file, index_label="City_ID")